In [1]:
import requests
import mimetypes
import uuid
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [2]:
# Google image search term
query = "chilidogs"

# Full search url with search term
url = f"https://www.google.com/search?tbm=isch&q={query}"

# Start Selenium to run our search
driver = webdriver.Chrome()
driver.get(url)

In [3]:
# Get the body element containing our search results
body = driver.find_element_by_css_selector('body')

# The following elements are used to scroll through results and find the end of the page
show_more_button = body.find_element_by_xpath('//input[@value="Show more results"]')
end_of_page = body.find_element_by_xpath('//div[text() = "Looks like you\'ve reached the end"]')

# Scroll to bottom of search results - this is done because not all images are loaded
while not show_more_button.is_displayed() or not end_of_page.is_displayed():
    body.send_keys(Keys.PAGE_DOWN)
    if show_more_button.is_displayed():
        show_more_button.click()
    if end_of_page.is_displayed():
        break

# Extract potential image urls
imgs = body.find_elements_by_class_name("rg_i")

# Keep only items with "https://" in the src attribute
img_urls = [
    img.get_attribute('src') for img in imgs
    if img.get_attribute('src')
    and "https://" in img.get_attribute('src')
]

# Close out the browser as we no longer need it for the heavy lifting
driver.close()

In [4]:
# Save images to this directory
data_directory = "./data/google_images/"

# Iterate through our image urls and save images locally
for url in img_urls:
    print(f"Downloading {url}")
    resp = requests.get(url)
    mime_type = resp.headers.get('Content-Type')
    extension = mimetypes.guess_extension(mime_type)
    filename = uuid.uuid5(uuid.NAMESPACE_URL, url).hex
    with open(f"{data_directory}{filename}{extension}", "wb") as f:
        f.write(resp.content)